# Cobra Motor Mapping
This notebook runs the cobras through ROM in 3 non-colliding groups in order to create controlled step motor maps for the module.

In [2]:
from importlib import reload
import numpy as np

from ics.cobraCharmer import pfi as pfiControl

<module 'ics.cobraCharmer.pfi' from '/Users/kadofong/work/pfscobras/ics_cobraCharmer/python/ics/cobraCharmer/pfi.py'>

In [3]:
niter=5

In [4]:
mod1Cobras = pfiControl.PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = pfiControl.PFI.allocateCobraList([(1,30)])    

# partition module 1 cobras into non-interfering sets
moduleCobras = {}
for group in 1,2,3:
    cm = range(group,58,3)
    mod = [1]*len(cm)
    
    moduleCobras[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))

In [5]:
pfi = pfiControl.PFI(fpgaHost='localhost', doLoadModel=False) #'fpga' for real device.
#pfi.setFreq(allCobras)
pfi.loadModel('/Users/kadofong/work/pfscobras/jpltrip/data/updatedmap4.xml')

In [7]:
# Stage 2 testing
pfi.homePhi ( cobra_group, dir='cw' ) # stage 2 motor to CW hardstop
for cobra_group in moduleCobras.values():
    for idx in range(niter):
        pfi.homePhi ( cobra_group, dir='ccw' ) # stage 2 motor to CCW hardstop
        pfi.homePhi ( cobra_group, dir='cw' ) # stage 2 to CW hardstop

In [ ]:
# Stage 1 testing
pfi.homeTheta ( cobra_group, dir='cw' ) # stage 1 to CW hardstop
for cobra_group in moduleCobras.values():
    pfi.moveAllSteps ( cobra_group, (0,1000), dirs=('ccw','ccw')) # move out stage 2 motor (CCW)
    for idx in range(niter):
        pfi.homeTheta ( cobra_group, dir='ccw' ) # stage 1 to CCW hardstop
        pfi.homeTheta ( cobra_group, dir='cw' ) # stage 1 to CW hardstop
    pfi.homePhi ( cobra_group, dir='cw' ) # stage 2 to CW 